In [1]:
import os
# os.chdir('../')
%pwd

'c:\\Users\\akish.pothuri\\python\\LangChain\\experiment'

In [2]:
import pyodbc
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=GGKU4DELL1254;Database=projectcodefirst;Trusted_Connection=yes;')

In [3]:
cnxn.cursor()

In [4]:
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM Employees")

In [5]:
row = cursor.fetchall()

In [6]:
row

[(1, 'Madhu', 1), (2, 'Anil', 2), (3, 'Amir', 3), (7, 'Akbar', 3)]

In [ ]:
# pip install google-generativeai
#pip install python-dotenv

In [9]:
import os
from langchain.llms import GooglePalm
from dotenv import load_dotenv 
load_dotenv()


True

In [10]:
palm_key = os.getenv('api_key')

In [13]:
llm = GooglePalm(google_api_key=palm_key, temperature=0.3)

In [14]:
code = llm("Code to linear regression in machine learning algorithms")

c:\Users\akish.pothuri\python\LangChain\experiment\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [15]:
print(code)

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data
x = np.linspace(0, 10, 100)
y = 2 * x + 5 + np.random.normal(0, 5, 100)

# Plot data
plt.scatter(x, y)

# Fit linear regression model
w, b = np.linalg.lstsq(x.reshape(-1, 1), y.reshape(-1, 1))[0]

# Plot linear regression line
y_pred = w * x + b
plt.plot(x, y_pred, color='red')

plt.show()
```


In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine

In [17]:
# check the local ODBC driver and make sure it match with the traget database instance
import pyodbc
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [18]:
#windows authentication connection string.
db = SQLDatabase.from_uri(
    database_uri="mssql+pyodbc://GGKU4DELL1254/shopping?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [19]:
sql_toolkit=SQLDatabaseToolkit(db=db,llm=llm)
sql_toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019B20E4B670>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019B20E4B670>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019B20E4B670>),
 QuerySQLCheckerTool(description='Use this tool to 

In [20]:
print(db.get_table_names())

['Customer', 'OrderItem', 'Orders', 'Product', 'Supplier']


c:\Users\akish.pothuri\python\LangChain\experiment\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `get_table_names` was deprecated in LangChain 0.0.1 and will be removed in 0.2.0. Use get_usable_table_name instead.
  warn_deprecated(


In [22]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("what are the total number of customers")



> Entering new SQLDatabaseChain chain...
what are the total number of customers
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT COUNT(*) FROM Customer
SQLResult: [(91,)]
Answer:91
> Finished chain.


In [23]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns2 = db_chain("what are the names of the customers from germany courntry?")



> Entering new SQLDatabaseChain chain...
what are the names of the customers from germany courntry?
SQLQuery:SELECT FirstName, LastName FROM Customer WHERE Country = 'Germany'
SQLResult: [('Maria', 'Anders'), ('Hanna', 'Moos'), ('Sven', 'Ottlieb'), ('Peter', 'Franken'), ('Philip', 'Cramer'), ('Renate', 'Messner'), ('Alexander', 'Feuer'), ('Henriette', 'Pfalzheim'), ('Horst', 'Kloss'), ('Karin', 'Josephs'), ('Rita', 'Müller')]
Answer:Maria Anders, Hanna Moos, Sven Ottlieb, Peter Franken, Philip Cramer, Renate Messner, Alexander Feuer, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Rita Müller
> Finished chain.


In [24]:
qns3 = db_chain("what is the name of Supplier providing more products?")




> Entering new SQLDatabaseChain chain...
what is the name of Supplier providing more products?
SQLQuery:SELECT CompanyName FROM Supplier S JOIN Product P ON S.Id = P.SupplierId GROUP BY CompanyName ORDER BY COUNT(*) DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
SQLResult: [('Pavlova, Ltd.',)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Pavlova, Ltd.
> Finished chain.


In [25]:
qns3['result']

'Pavlova, Ltd.'

In [68]:
qns4 = db_chain("what is the name of customer ordered maximum quantity according the data in the database?")



> Entering new SQLDatabaseChain chain...
what is the name of customer ordered maximum quantity according the data in the database?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT TOP 1 c.FirstName + ' ' + c.LastName AS CustomerName
FROM Customer AS c
INNER JOIN Orders AS o ON c.Id = o.CustomerId
INNER JOIN OrderItem AS oi ON o.Id = oi.OrderId
GROUP BY o.CustomerId
ORDER BY SUM(oi.Quantity) DESC

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.FirstName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.LastName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120)")
[SQL: SELECT TOP 1 c.FirstName + ' ' + c.LastName AS CustomerName
FROM Customer AS c
INNER JOIN Orders AS o ON c.Id = o.CustomerId
INNER JOIN OrderItem AS oi ON o.Id = oi.OrderId
GROUP BY o.CustomerId
ORDER BY SUM(oi.Quantity) DESC]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [27]:
#Name of the person and country who ordered more quantity of products.
db_chain.run("SELECT TOP (1) CU.FirstName +' '+ CU.LastName,CU.Country FROM Orders as O INNER JOIN OrderItem as OI ON O.Id=OI.OrderId INNER JOIN Customer AS CU ON CU.Id = O.CustomerId ORDER BY Quantity DESC;")



> Entering new SQLDatabaseChain chain...
SELECT TOP (1) CU.FirstName +' '+ CU.LastName,CU.Country FROM Orders as O INNER JOIN OrderItem as OI ON O.Id=OI.OrderId INNER JOIN Customer AS CU ON CU.Id = O.CustomerId ORDER BY Quantity DESC;
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT TOP (1) CU.FirstName +' '+ CU.LastName,CU.Country FROM Orders as O INNER JOIN OrderItem as OI ON O.Id=OI.OrderId INNER JOIN Customer AS CU ON CU.Id = O.CustomerId ORDER BY Quantity DESC;
SQLResult: [('Roland Mendel', 'Austria')]
Answer:Roland Mendel, Austria
> Finished chain.


'Roland Mendel, Austria'

In [66]:
qns4 = db_chain("Name of the person who ordered maximum quantity of products till now")



> Entering new SQLDatabaseChain chain...
Name of the person who ordered maximum quantity of products till now
SQLQuery:SELECT TOP 1 FirstName + ' ' + LastName AS CustomerName 
FROM Customer AS C JOIN Orders AS O ON C.Id = O.CustomerId 
JOIN OrderItem AS I ON O.Id = I.OrderId 
GROUP BY C.Id 
ORDER BY SUM(I.Quantity) DESC

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.FirstName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.LastName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120)")
[SQL: SELECT TOP 1 FirstName + ' ' + LastName AS CustomerName 
FROM Customer AS C JOIN Orders AS O ON C.Id = O.CustomerId 
JOIN OrderItem AS I ON O.Id = I.OrderId 
GROUP BY C.Id 
ORDER BY SUM(I.Quantity) DESC]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [31]:
# Name of the person who ordered maximum quantity of products till now
qns4 = db_chain.run("  select Top 1 CU.FirstName +' '+CU.LastName from OrderItem as OI join Orders as O on OI.OrderId = O.Id join Customer as CU on O.CustomerId = CU.Id order by OI.Quantity DESC")



> Entering new SQLDatabaseChain chain...
  select Top 1 CU.FirstName +' '+CU.LastName from OrderItem as OI join Orders as O on OI.OrderId = O.Id join Customer as CU on O.CustomerId = CU.Id order by OI.Quantity DESC
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT TOP 1 CU.FirstName +' '+CU.LastName FROM OrderItem AS OI JOIN Orders AS O ON OI.OrderId = O.Id JOIN Customer AS CU ON O.CustomerId = CU.Id ORDER BY OI.Quantity DESC
SQLResult: [('Roland Mendel',)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Roland Mendel
> Finished chain.


In [59]:
qns5 = db_chain("Name of the customers country wise. give answer in json format?")



> Entering new SQLDatabaseChain chain...
Name of the customers country wise. give answer in json format?
SQLQuery:SELECT Customer.Country, Customer.FirstName + ' ' + Customer.LastName AS CustomerName
FROM Customer
GROUP BY Customer.Country
ORDER BY Customer.Country

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.FirstName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column 'Customer.LastName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120)")
[SQL: SELECT Customer.Country, Customer.FirstName + ' ' + Customer.LastName AS CustomerName
FROM Customer
GROUP BY Customer.Country
ORDER BY Customer.Country]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [32]:
qns5 = db_chain.run("SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;")



> Entering new SQLDatabaseChain chain...
SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName
SQLResult: [('Argentina', 'Patricio Simpson'), ('Argentina', 'Sergio Gutiérrez'), ('Argentina', 'Yvonne Moncada'), ('Austria', 'Georg Pipps'), ('Austria', 'Roland Mendel'), ('Belgium', 'Catherine Dewey'), ('Belgium', 'Pascale Cartrain'), ('Brazil', 'Anabela Domingues'), ('Brazil', 'André Fonseca'), ('Brazil', 'Aria Cruz'), ('Brazil', 'Bernardo Batista'), ('Brazil', 'Janete Limeira'), ('Brazil', 'Lúcia Carvalho'), ('Brazil', 'Mario Pontes'), ('Brazil', 'Paula Parente'), ('Brazil', 'Pedro Afonso'), ('Canada', 'Elizabeth Lincoln'), ('Canada', 'Jean Fresnière'), ('Canada', 'Yoshi Tannamuri'), ('Denmark', 'Jytte Petersen'), ('Denmark', 'Palle Ibsen'), ('Finland', 'Matti Karttunen'), ('Finland', 'Pirkko Koskitalo'), ('France', 'Annette Roulet'), ('France', 'Carine Schmitt'), ('France', 'Daniel Tonini'), ('France', 'Dominique Perrier'), ('France', 'Frédérique Citeaux'), ('France', 'Janine Labrune'),

In [29]:
qns6 = db_chain("Give me the date on we got maximum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got maximum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT OrderDate FROM Orders ORDER BY TotalAmount DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
SQLResult: [(datetime.datetime(2014, 2, 2, 0, 0),)]
Answer:2014-02-02
> Finished chain.


In [30]:
qns4 = db_chain("Give me the date on we got minimum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got minimum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT MIN(OrderDate) FROM Orders
SQLResult: [(datetime.datetime(2012, 7, 4, 0, 0),)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


2012-07-04
> Finished chain.
